In [1]:
import os
import sys
import json
import joblib
import geopandas as gpd
import torch
import help_functions as hf

current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir)
sys.path.append(parent_dir)
from gnn_architecture import MyGnn  # or whatever you need to import
import help_functions as hf

<!-- ABSTRACT -->

THIS IS WORK IN PROGRESS! 

The goal of this script is to apply monte carlo dropout to the trained model and check how well it performs.

Perspectively, one can also introduce more uncertainty methods. 

In [2]:
# Parameters to adapt

run_path = '../../data/runs_01_2025/'
point_net_conv_layer_structure_local_mlp = "256"
point_net_conv_layer_structure_global_mlp = "512"
gat_conv_layer_structure = "128,256,512,256" 
dropout = 0.3
use_dropout = False 
predict_mode_stats = False
in_channels = 5
out_channels = 1 
loss_fct = torch.nn.MSELoss()

# The rest we can usually leave as is
districts = gpd.read_file("../../data/visualisation/districts_paris.geojson")
pnc_l_string = point_net_conv_layer_structure_local_mlp.replace(',', '_')
pnc_g_string = point_net_conv_layer_structure_global_mlp.replace(',', '_') 
gat_string = gat_conv_layer_structure.replace(',', '_')

# unique_model_description = f"pnc_local_[{pnc_l_string}]_" + \
# f"pnc_global_[{pnc_g_string}]_" + \
# f"gat_conv_[{gat_string}]_" + \
# f"use_dropout_{use_dropout}_" + \
# f"dropout_{dropout}_" + \
# f"predict_mode_stats_{predict_mode_stats}" + "/"

unique_model_description = "single_districts_1pct/"
        
run_path = run_path + unique_model_description
base_case_path = '../../data/test_data/pop_1pm_basecase_mean_links_NEW.geojson'
links_base_case = gpd.read_file(base_case_path, crs="EPSG:4326")
model_path = run_path +  'trained_model/model.pth'
data_created_during_training = run_path + 'data_created_during_training/'
scaler_x = joblib.load(data_created_during_training + 'test_x_scaler.pkl')
scaler_pos = joblib.load(data_created_during_training + 'test_pos_scaler.pkl')

In [3]:
# Load the test dataset created during training
test_set_dl = torch.load(data_created_during_training + 'test_dl.pt')

# Load the DataLoader parameters
with open(data_created_during_training + 'test_loader_params.json', 'r') as f:
    test_set_dl_loader_params = json.load(f)
    
# Remove or correct collate_fn if it is incorrectly specified
if 'collate_fn' in test_set_dl_loader_params and isinstance(test_set_dl_loader_params['collate_fn'], str):
    del test_set_dl_loader_params['collate_fn']  # Remove it to use the default collate function
    
test_set_loader = torch.utils.data.DataLoader(test_set_dl, **test_set_dl_loader_params)

In [4]:
point_net_conv_layer_structure_local_mlp = [int(x) for x in point_net_conv_layer_structure_local_mlp.split(',')]
point_net_conv_layer_structure_global_mlp = [int(x) for x in point_net_conv_layer_structure_global_mlp.split(',')]
gat_conv_layer_structure = [int(x) for x in gat_conv_layer_structure.split(',')]

model = MyGnn(in_channels=in_channels, out_channels=out_channels, 
                    point_net_conv_layer_structure_local_mlp=point_net_conv_layer_structure_local_mlp, 
                    point_net_conv_layer_structure_global_mlp = point_net_conv_layer_structure_global_mlp,
                    gat_conv_layer_structure=gat_conv_layer_structure,
                    dropout=dropout,
                    use_dropout=use_dropout, 
                    predict_mode_stats=predict_mode_stats)

# Load the model state dictionary
model.load_state_dict(torch.load(model_path))

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

/opt/anaconda3/envs/chenhao-gnn/lib/python3.10/site-packages/torch/nn/modules/transformer.py:286: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


Model initialized
MyGnn(
  (point_net_conv_1): PointNetConv(local_nn=Sequential(
    (0): Linear(in_features=7, out_features=256, bias=True)
    (1): ReLU()
  ), global_nn=Sequential(
    (0): Linear(in_features=256, out_features=512, bias=True)
    (1): Linear(in_features=512, out_features=512, bias=True)
    (2): ReLU()
  ))
  (point_net_conv_2): PointNetConv(local_nn=Sequential(
    (0): Linear(in_features=514, out_features=256, bias=True)
    (1): ReLU()
  ), global_nn=Sequential(
    (0): Linear(in_features=256, out_features=512, bias=True)
    (1): Linear(in_features=512, out_features=128, bias=True)
    (2): ReLU()
  ))
  (read_out_node_predictions): Linear(in_features=64, out_features=1, bias=True)
  (gat_graph_layers): Sequential(
    (0) - TransformerConv(128, 64, heads=4): x, edge_index -> x
    (1) - ReLU(inplace=True): x -> x
    (2) - TransformerConv(256, 128, heads=4): x, edge_index -> x
    (3) - ReLU(inplace=True): x -> x
    (4) - TransformerConv(512, 64, heads=4): x,

In [5]:
test_loss_my_test_data, r_squared_my_test_data, actual_vals_my_test_data, predictions_my_test_data, baseline_loss_my_test_data = hf.validate_model_on_test_set(model, test_set_loader.dataset, loss_fct, device)

print(f"Test Loss: {test_loss_my_test_data}")
print(f"R-squared: {r_squared_my_test_data}")
print(f"Baseline Loss: {baseline_loss_my_test_data}")

Test Loss: 260.02290836035036
R-squared: 0.048076331615448
Baseline Loss: 273.1551818847656


# Next, we will look at single elements of the test set and visualize the performance of the model.


In [16]:
fixed_norm_max = 8

i = 0

my_test_data = test_set_loader.dataset[i]
my_test_x = test_set_loader.dataset[i].x
my_test_x = my_test_x.to('cpu')

test_loss_my_test_data, r_squared_my_test_data, actual_vals_my_test_data, predictions_my_test_data, baseline_loss_my_test_data = hf.validate_model_on_test_set(model, my_test_data, loss_fct, device)
print(f"Test {i}")
print(f"Test Loss: {test_loss_my_test_data}")
print(f"R-squared: {r_squared_my_test_data}")
print(f"Baseline Loss: {baseline_loss_my_test_data}")

inversed_x = scaler_x.inverse_transform(my_test_x)

print(f"Length of original_gdf: {len(links_base_case)}")
print(f"Length of inversed_x: {inversed_x.shape[0]}")  # Assuming inversed_x is a numpy array

# AKTUELL STIMMEN DIE GROESSEN DER BEIDEN GDFS NICHT UEBEREIN. DIES FIXEN WENN READY. 

gdf_with_og_values = hf.data_to_geodataframe_with_og_values(data=my_test_data, original_gdf=links_base_case, predicted_values=predictions_my_test_data, inversed_x=inversed_x, use_all_features=False)
gdf_with_og_values['capacity_reduction_rounded'] = gdf_with_og_values['capacity_reduction'].round(decimals=3)
gdf_with_og_values['highway'] = gdf_with_og_values['highway'].map(hf.highway_mapping)

Test 0
Test Loss: 2.0392765317644392
R-squared: 0.01927924156188965
Baseline Loss: 14.55555534362793
Length of original_gdf: 31140
Length of inversed_x: 31635


In [18]:
my_test_x.shape[0]

31635

In [ ]:
# DIES KANN EIG AUCH RAUS

hf.plot_combined_output(gdf_input=gdf_with_og_values, column_to_plot="vol_car_change_predicted", 
                        save_it=True, number_to_plot=i, zone_to_plot = "this zone", is_predicted=True, alpha=0, use_fixed_norm=True, 
                        fixed_norm_max = fixed_norm_max,
                        known_districts = False, buffer = 0.0005, districts_of_interest=None)
hf.plot_combined_output(gdf_input=gdf_with_og_values, column_to_plot="vol_car_change_actual", save_it=True, 
                        number_to_plot=i, zone_to_plot = "this zone",is_predicted=False,alpha=10,use_fixed_norm=True, 
                        fixed_norm_max = fixed_norm_max,
                        known_districts = False, buffer = 0.0005, districts_of_interest=None)

In [24]:
# DO MC DROPOUT AFTERWARDS
mean_predictions, uncertainties = mc_dropout_predict(model, test_data, num_samples=50, device=device)

tensor([[ 0.0021],
        [ 0.0021],
        [ 0.0031],
        ...,
        [-0.0001],
        [ 0.0004],
        [ 0.0006]], device='cuda:0')